In [1]:
import pandas as pd
import numpy as np
import io
import os
import re
from google.colab import drive
!pip install transformers
!pip install pickle5
from transformers import pipeline
drive.mount('/content/drive')

     |████████████████████████████████| 2.3MB 6.5MB/s 
     |████████████████████████████████| 901kB 31.4MB/s 
     |████████████████████████████████| 3.3MB 42.4MB/s 
     |████████████████████████████████| 133kB 6.5MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219282 sha256=9cbabd3d7387bc3160b84cd977ef9128aad73c265a2b4c0199be5c224fe02221
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5
Mounted at /content/drive


In [2]:
data_path = '/content/drive/MyDrive/colab_data'
processed_letter_df = pd.read_csv(os.path.join(data_path, 'processed_letter.csv'))
processed_letter_df['tokens'] = processed_letter_df.clean_letter.apply(lambda x: x.split(' '))

## Transformers sentiment analysis

In [3]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

In [8]:
top10freq_words = ['earnings', 'insurance', 'shareholder', 'stock', 'loss', 'gain', 'profit', 'increase', 'sell', 'rate']
df = pd.DataFrame(columns=['text', 'sentiment', 'score'])
for w in top10freq_words:
  result = classifier(w)
  new_row = {'text': w, 'sentiment':result[0]['label'], 'score':result[0]['score']}
  df = df.append(new_row, ignore_index=True)
df.head(3)

text sentiment     score
0     earnings  POSITIVE  0.999316
1    insurance  POSITIVE  0.993685
2  shareholder  POSITIVE  0.997295

## Transformers QA

In [14]:
import pickle5 as pickle
with open(os.path.join(data_path, 'letters_dict.pickle'), 'rb') as handle:
    letters_dict = pickle.load(handle)

In [15]:
def get_answer_using_qa(nlp, question, context):
    """Get answer using a classifier trained with the QA technique
    Parameters
    ----------
    nlp: Pipeline
        Trained QA Pipeline
    question: String
        Question that the model will answer
    context: String
        The Context of the question
    Returns
    -------
    Tuple
        The answer, the score, the start position of the answer at the text and the final
        position of the answer at the text
    """
    result = nlp(question=question, context=context)

    return result['answer'], round(result['score'], 4), result['start'], result['end']

In [17]:
nlp = pipeline("question-answering")


In [18]:
print(sentence_after_regex_2008)

Berkshire sCorporatePerformancevs theS P500 AnnualPercentageChange inPer Share inS P500 BookValueof withDividends Relative Berkshire Included Results Year  1   2   1   2  1965                                                     23 8 10 0 13 8 1966                                                     20 3  11 7  32 0 1967                                                     11 0 30 9  19 9  1968                                                     19 0 11 0 8 0 1969                                                     16 2  8 4  24 6 1970                                                     12 0 3 9 8 1 1971                                                     16 4 14 6 1 8 1972                                                     21 7 18 9 2 8 1973                                                     4 7  14 8  19 5 1974                                                     5 5  26 4  31 9 1975                                                     21 9 37 2  15 3  1976                             

In [26]:
year = 2020
sentence_after_regex = re.sub("[^a-z0-9A-Z\,]", " ", letters_dict[year])
answer, result, start, end = get_answer_using_qa(nlp, "what's the biggest profit this year?", sentence_after_regex)
print(f"Answer: '{answer}', score: {result}, start: {start}, end: {end}")

Answer: '0 93  at yearend   had fallen 94', score: 0.0794, start: 13656, end: 13688


## Transformer embedding

In [4]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 1.2MB 8.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=17639dd31e137ba1c8ef8285f8ead5f2672644d2d1929ad77c3fabab7863b97b
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers


In [ ]:
model = SentenceTransformer()